In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
training_data = pd.read_csv("train.csv")
testing_data = pd.read_csv("test.csv")

In [6]:
training_data.sample(2)

,id,keyword,location,text,target
3869,5501,flames,? Jet Life ?,I have a rising follower graph! 2 more followe...,0
2884,4145,drought,NaN,LLF TALK WORLD NEWS U.S. in record hurricane ...,1


In [7]:
training_data['text'].isnull().sum()

0

In [8]:
training_data['new_text'] = training_data['text'].apply(lambda x:x.lower())

In [9]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()

In [11]:
from nltk.corpus import stopwords

In [10]:
training_data['new_text'] = training_data['new_text'].apply(lambda x:lemm.lemmatize(x))

In [14]:
stop_words = stopwords.words("english")

In [18]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [16]:
def remove_stopwords(text):
    no_stop = []
    for word in text.split(' '):
        if word not in stop_words:
            no_stop.append(word)
    return " ".join(no_stop)

In [17]:
training_data['new_text'] = training_data['new_text'].apply(remove_stopwords)

In [20]:
import re

In [23]:
def clean_text(text):
    text = re.sub(r'^RT[\s]+', '', text)
    text = re.sub(r'https?://[^\s\n\r]+', '', text)
    text = re.sub(r'#', '', text)
    return text

In [24]:
training_data['new_text'] = training_data['new_text'].apply(clean_text)

In [25]:
training_data.sample(2)

,id,keyword,location,text,target,new_text
2849,4095,displaced,Palestine,@cityofhummus @ILNewsFlash do you want to hear...,0,@cityofhummus @ilnewsflash want hear more? dis...
3424,4897,explode,"Cleveland, TN",VINE OF THE YEAR OH MY GOD I AM ABOUT TO EXPLO...,0,vine year oh god explode


In [26]:
def remove_punctuation_func(text):
    '''
    Removes all punctuation from a string, if present
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string without punctuations
    '''
    return re.sub(r'[^a-zA-Z0-9]', ' ', text)

In [27]:
training_data['new_text'] = training_data['new_text'].apply(remove_punctuation_func)

In [28]:
training_data.head()

,id,keyword,location,text,target,new_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified offic...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 people receive wildfires evacuation ord...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [81]:
X = training_data['new_text']

In [82]:
y = training_data['target']

In [83]:
np.unique(y)

array([0, 1])

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [86]:
token = Tokenizer(num_words=10000,oov_token="<OOV>")

In [87]:
token.fit_on_texts(X) #only once- that is on training

In [88]:
word_index = token.word_index

In [89]:
training_seq = token.texts_to_sequences(X)

In [90]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [91]:
train_padded = pad_sequences(training_seq,padding="post",truncating="post",maxlen=50)

In [92]:
X_train,X_test,y_train,y_test = train_test_split(train_padded,y,test_size=0.2,train_size=0.8)

In [93]:
from tensorflow import keras
#import tensorflow.keras as keras

In [97]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10000,128))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1,activation="sigmoid"))

In [98]:
model.compile("rmsprop", "binary_crossentropy", metrics=["accuracy"])

In [99]:
history = model.fit(X_train,y_train,epochs=2,validation_data=(X_test,y_test))

Epoch 1/2
191/191 [==============================] - 20s 64ms/step - loss: 0.5371 - accuracy: 0.7328 - val_loss: 0.4314 - val_accuracy: 0.8043
Epoch 2/2
191/191 [==============================] - 11s 59ms/step - loss: 0.3653 - accuracy: 0.8504 - val_loss: 0.4392 - val_accuracy: 0.8109
